In [1]:
!pip install pandas
!pip install sklearn

In [2]:
import pandas as pd 
import sys
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier

import classifier_tool as tool

In [3]:
label = 'angin_topan'
rep_list = [
    't_lower', 'c_lower', 'tc_lower',
    't_swrem', 'c_swrem', 'tc_swrem',
    't_stem', 'c_stem', 'tc_stem',
    't_swrem_stem', 'c_swrem_stem', 'tc_swrem_stem',
]
#f_in = '/home/admin/text_processing/anotated_data/preprocessed/' + label + '_anotated.json'
#f_in = '/home/admin/text_processing/anotated_data/from_anotators/yandi/sample_angin_topan_20200414_yandi_20200417_fixed_pre.json'
f_in = '/home/admin/text_processing/anotated_data/from_anotators/ozy/sample_angin_topan_20200414_ozy_20200421_fixed_pre.json'
df = pd.read_json(f_in)

In [4]:
df

,id,source,date,title,label,t_lower,c_lower,tc_lower,t_swrem,c_swrem,tc_swrem,t_stem,c_stem,tc_stem,t_swrem_stem,c_swrem_stem,tc_swrem_stem
0,angin_topan_0001,aceh_harianrakyat,"October 1, 2018",Puting Beliung Rusak Sembilan Rumah,Y,puting beliung rusak sembilan rumah,redelong (ra) puting beliaung kembali terjad...,puting beliung rusak sembilan rumah. redelong ...,puting beliung rusak sembilan rumah,redelong (ra) puting beliaung terjadi bener ...,puting beliung rusak sembilan rumah. redelong ...,puting beliung rusak sembilan rumah,redelong ra puting beliaung kembali jadi di be...,puting beliung rusak sembilan rumah. redelong ...,puting beliung rusak sembilan rumah,redelong ra puting beliaung jadi bener riah ak...,puting beliung rusak sembilan rumah. redelong ...
1,angin_topan_0002,aceh_harianrakyat,"November 11, 2016",Pencarian Nelayan Simeulue Dihentikan,Y,pencarian nelayan simeulue dihentikan,"simeulue (ra) setelah sepekan operasi, penca...",pencarian nelayan simeulue dihentikan. simeulu...,pencarian nelayan simeulue dihentikan,"simeulue (ra) sepekan operasi, pencarian nel...",pencarian nelayan simeulue dihentikan. simeulu...,cari nelayan simeulue henti,simeulue ra telah pek operasi cari dua nelayan...,cari nelayan simeulue henti. simeulue ra telah...,cari nelayan simeulue henti,simeulue ra pek operasi cari nelayan simeulue ...,cari nelayan simeulue henti. simeulue ra pek o...
2,angin_topan_0003,aceh_harianrakyat,"January 4, 2018",Pohon Tumbang Timpa Warga Korban Meninggal Dal...,y,pohon tumbang timpa warga korban meninggal dal...,sabang (ra) seorang pengendara motor dilapor...,pohon tumbang timpa warga korban meninggal dal...,pohon tumbang timpa warga korban meninggal kap...,sabang (ra) seorang pengendara motor dilapor...,pohon tumbang timpa warga korban meninggal kap...,pohon tumbang timpa warga korban tinggal dalam...,sabang ra orang kendara motor lapor alami luka...,pohon tumbang timpa warga korban tinggal dalam...,pohon tumbang timpa warga korban tinggal kapal...,sabang ra orang kendara motor lapor alami luka...,pohon tumbang timpa warga korban tinggal kapal...
3,angin_topan_0004,aceh_harianrakyat,"February 11, 2020",Ulama Tetap Menjadi Referensi KPA/PA,N,ulama tetap menjadi referensi kpa/pa,meulaboh (ra) ulama tetap menjadi panutan da...,ulama tetap menjadi referensi kpa/pa. meulaboh...,ulama tetap menjadi referensi kpa/pa,meulaboh (ra) ulama tetap menjadi panutan pe...,ulama tetap menjadi referensi kpa/pa. meulaboh...,ulama tetap jadi referensi kpa pa,meulaboh ra ulama tetap jadi panutan dan tunju...,ulama tetap jadi referensi kpa pa. meulaboh ra...,ulama tetap jadi referensi kpa pa,meulaboh ra ulama tetap jadi panutan tunjuk la...,ulama tetap jadi referensi kpa pa. meulaboh ra...
4,angin_topan_0005,aceh_harianrakyat,"April 10, 2017",Empat Rumah Warga Rusak Dihantam Puting Beliung,Y,empat rumah warga rusak dihantam puting beliung,simeulue (ra) sebanyak empat rumah warga des...,empat rumah warga rusak dihantam puting beliun...,empat rumah warga rusak dihantam puting beliung,simeulue (ra) sebanyak empat rumah warga des...,empat rumah warga rusak dihantam puting beliun...,empat rumah warga rusak hantam puting beliung,simeulue ra banyak empat rumah warga desa amai...,empat rumah warga rusak hantam puting beliung....,empat rumah warga rusak hantam puting beliung,simeulue ra banyak empat rumah warga desa amai...,empat rumah warga rusak hantam puting beliung....
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,angin_topan_1206,surabaya_tribun,"Jumat, 13 Desember 2019 16:33",Pemkot Surabaya Waspadai Potensi Bencana Akiba...,N,pemkot surabaya waspadai potensi bencana akiba...,"laporan wartawan tribunjatim. com, yusron nauf...",pemkot surabaya waspadai potensi bencana akiba...,pemkot surabaya waspadai potensi bencana akiba...,"laporan wartawan tribunjatim. com, yusron nauf...",pemkot surabaya waspadai potensi bencana akiba...,pemkot surabaya waspada potensi bencana akibat...,lapor wartawan tribunjati

In [5]:
def test_rep(rep, df, clf, fold=10):
    X = tool.construct_tfidf_unigrams(df[rep])
    y = df['label']
    report = tool.eval_cv(fold, X, y, clf)
    y_report = [ d['Y'] for d in report ]
    n_report = [ d['N'] for d in report ]
    
    df_y = pd.DataFrame(y_report)
    df_n = pd.DataFrame(n_report)
    df_y.loc['avg'] = df_y.mean()
    df_n.loc['avg'] = df_n.mean()
    df_y_avg = df_y.loc[['avg']]
    df_y_avg = df_y_avg.rename(columns={'precision' : 'y_prec', 'recall':'y_recall', 'f1-score' : 'y_f1', 'support' : 'y_support'})
    
    return df_y_avg, df_y, df_n
    
    

In [6]:
clf = SVC()
df_summary = pd.DataFrame()

for rep in rep_list:
    df_y_avg, df_y, df_n = test_rep(rep_list[0], df, clf)
    df_summary[rep] = df_y_avg.loc['avg']

In [7]:
df_summary

,t_lower,c_lower,tc_lower,t_swrem,c_swrem,tc_swrem,t_stem,c_stem,tc_stem,t_swrem_stem,c_swrem_stem,tc_swrem_stem
y_prec,0.863336,0.852159,0.884870,0.863868,0.878281,0.859225,0.867320,0.871104,0.869724,0.862779,0.853840,0.869178
y_recall,0.444544,0.438121,0.424424,0.443908,0.457274,0.439984,0.422512,0.453378,0.439907,0.436662,0.413835,0.432732
y_f1,0.583361,0.572951,0.559564,0.578308,0.595130,0.565485,0.556008,0.588250,0.581077,0.578112,0.549866,0.574704
y_support,26.800000,26.800000,26.800000,26.800000,26.800000,26.800000,26.800000,26.800000,26.800000,26.800000,26.800000,26.800000
